In [ ]:
from download_pdf.database_preparation import combine_scopus_csvs_to_excel
from setting.project_path import project_folder
import os
project_review='wafer_defect'
path_dic=project_folder(project_review=project_review)
folder_path =os.path.join(path_dic['main_folder'],'database','scopus')
    # r"C:\Users\balan\OneDrive - ums.edu.my\research_related\corona_discharge\database\scopus"
output_excel = os.path.join(path_dic['main_folder'],'database','wafer_database.xlsx')
    # r"C:\Users\balan\OneDrive - ums.edu.my\research_related\corona_discharge\database\combined_filtered.xlsx"

combine_scopus_csvs_to_excel(folder_path, output_excel)


# Filter Database

In [ ]:
from research_filter.auto_llm import run_pipeline
# project_review='eeg_review'
project_review='wafer_defect'
path_dic=project_folder(project_review=project_review)
main_folder = path_dic['main_folder']

# Editable variables
# model_name = "gpt-4o"  # or "gpt-4o"
model_name="gpt-4o-mini"
# model_name="gpt-o3-mini"
# model_name='gemini-1.5-pro'

# model_name='gemini-exp-1206'
# model_name='gemini-2.0-flash-thinking-exp-01-21'


agentic_setting = {
    "agent_name": "abstract_wafer_abstract_filter",
    "column_name": "abstract_wafer_abstract_filter",
    "yaml_path": "agent/agent_ml.yaml",
    "model_name": model_name
}

methodology_json_folder=os.path.join(main_folder,agentic_setting['agent_name'],'json_output')
multiple_runs_folder =os.path.join(main_folder,agentic_setting['agent_name'],'multiple_runs_folder')
final_cross_check_folder = os.path.join(main_folder,agentic_setting['agent_name'],'final_cross_check_folder')

csv_path = path_dic['csv_path']
# Basic placeholders for roles
placeholders = {} # we just ignore this as im to lazy to generalize things


process_setup={
    'batch_process':False,  # This is for the batch processing which have 50% discount. use with the code research_filter/check_batch_process.py
    'manual_paste_llm':False,       # This is for the manual paste of the LLM
    'iterative_confirmation':False,
    'overwrite_csv':False,      # careful when set to true as this will overwrite the csv file
    'cross_check_enabled':False,
    'cross_check_runs':3,
    'cross_check_agent_name':'agent_cross_check',
    'cleanup_json':False
}

# Run pipeline
run_pipeline(
    agentic_setting ,process_setup,
    placeholders=placeholders,
    csv_path=csv_path,
    main_folder=main_folder,
    methodology_json_folder=methodology_json_folder,
    multiple_runs_folder=multiple_runs_folder,
    final_cross_check_folder=final_cross_check_folder,
)



# Download pdf

In [ ]:
from download_pdf.download_pdf import run_pipeline,process_scihub_downloads,process_fallback_ieee,process_fallback_mdpi,process_fallback_sciencedirect

project_review='wafer_defect'
path_dic=project_folder(project_review=project_review)
main_folder = path_dic['main_folder']

# csv_path = path_dic['csv_path']
# pdf_store_directory = os.path.join(main_folder, 'pdf')


file_path = path_dic['csv_path']
output_folder =  os.path.join(main_folder, 'pdf')

categories,data_filtered=run_pipeline(file_path)

# First step, we will always use the scihub to download the pdf
process_scihub_downloads(categories, output_folder, data_filtered)

# Skipping the IEEE fallback step for testing
process_fallback_ieee(categories, data_filtered, output_folder)

# process_fallback_ieee_search(categories, data_filtered, output_folder)

# process_fallback_mdpi(categories, data_filtered, output_folder)

# The sciencedirect only allow to extract the url, but not able to download the pdf due to the tight security feature
# process_fallback_sciencedirect(categories, data_filtered, output_folder)
# save_data(data_filtered, file_path)
